In [ ]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# Apache Beam RunInference with TensorFlow

<button>
  <a href="https://beam.apache.org/documentation/sdks/python-machine-learning/">
    <img src="https://beam.apache.org/images/favicon.ico" alt="Open the docs" height="16"/>
    Beam RunInference
  </a>
</button>

The Apache Beam RunInference transform is used for making predictions for
a variety of machine learning models. From version 1.10.0 of tfx-bsl you can
create a TensorFlow ModelHandler for use with Apache Beam.

In this notebook, we walk through the use of the RunInference transform for [TensorFlow](https://www.tensorflow.org/).
Beam [RunInference](https://beam.apache.org/releases/pydoc/current/apache_beam.ml.inference.base.html#apache_beam.ml.inference.base.RunInference) accepts ModelHandler generated from [tfx-bsl](https://github.com/tensorflow/tfx-bsl) via CreateModelHandler.



In this notebook we walk through:
- Importing [tfx-bsl](https://github.com/tensorflow/tfx-bsl)
- Building a simple TensorFlow model
- Setting up example data into TensorFlow protos.
- Running those examples and getting a prediction inside an Apache Beam pipeline.

In [1]:
!pip install tfx_bsl==1.10.0 --quiet

     |████████████████████████████████| 21.6 MB 2.3 MB/s 
     |████████████████████████████████| 10.9 MB 18.8 MB/s 
     |████████████████████████████████| 578.0 MB 17 kB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 270 kB 71.2 MB/s 
     |████████████████████████████████| 151 kB 69.7 MB/s 
     |████████████████████████████████| 47 kB 6.0 MB/s 
     |████████████████████████████████| 2.4 MB 54.5 MB/s 
     |████████████████████████████████| 508 kB 73.5 MB/s 
     |████████████████████████████████| 267 kB 58.1 MB/s 
     |████████████████████████████████| 83 kB 2.4 MB/s 
     |████████████████████████████████| 255 kB 73.4 MB/s 
     |████████████████████████████████| 148 kB 72.0 MB/s 
     |████████████████████████████████| 180 kB 70.6 MB/s 
     |████████████████████████████████| 124 kB 61.7 MB/s 
     |████████████████████████████████| 173 kB 68.3 MB/s 
     |████████████████████████████████| 435 kB 59.1 MB/s 
     |█████████████

In order to use RunInference you will need beam 2.40 or higher. Creation of a ModelHandler is supported in tfx-bsl 1.10 or higher.

In [2]:
!pip freeze | grep beam
!pip freeze | grep tfx-bsl


apache-beam==2.41.0
tfx-bsl==1.10.0


## Authenticate With Cloud
This notebook relies on saving your model into Google Cloud. First authenticate this notebook to use your Google Cloud account.

In [3]:
from google.colab import auth
auth.authenticate_user()

## Import Your Dependencies and Setup Your Bucket
Replace project and bucket with your variables with your project.

**Important**: If you get an error, restart your runtime.

In [1]:
import argparse

import tensorflow as tf
from tensorflow import keras
from tensorflow_serving.apis import prediction_log_pb2

import apache_beam as beam
from apache_beam.ml.inference.base import RunInference
import tfx_bsl
from tfx_bsl.public.beam.run_inference import CreateModelHandler
from tfx_bsl.public import tfxio
from tfx_bsl.public.proto import model_spec_pb2
from tensorflow_metadata.proto.v0 import schema_pb2

import numpy

from typing import Dict, Text, Any, Tuple, List

from apache_beam.options.pipeline_options import PipelineOptions

project = "<Replace With Your Project>"
bucket = "<Replace With Your Bucket>" 

save_model_dir_multiply = f'gs://{bucket}/tfx-inference/model/multiply_five/v1/'


## Create and Test a Simple Model

This creates a model that predicts the 5 times table.

In [2]:
# Create training data which represents the 5 times multiplication table for 0 to 99. 
# x is the data and y the labels. 
x = numpy.arange(0, 100)   # Examples
y = x * 5                  # Labels

# Build a simple linear regression model.
# Note the model has a shape of (1) for its input layer, it will expect a single int64 value.
input_layer = keras.layers.Input(shape=(1), dtype=tf.float32, name='x')
output_layer= keras.layers.Dense(1)(input_layer)

model = keras.Model(input_layer, output_layer)
model.compile(optimizer=tf.optimizers.Adam(), loss='mean_absolute_error')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 x (InputLayer)              [(None, 1)]               0         
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


### Test the Model


In [4]:
model.fit(x, y, epochs=500, verbose=0)
test_examples =[20, 40, 60, 90]
value_to_predict = numpy.array(test_examples, dtype=numpy.float32)
predictions = model.predict(value_to_predict)

print('Test Examples ' + str(test_examples))
print('Predictions ' + str(predictions))

1/1 [==============================] - 0s 16ms/step
Test Examples [20, 40, 60, 90]
Predictions [[ 99.99883]
 [199.99783]
 [299.99686]
 [449.99533]]


### Populate the Data into a TensorFlow Proto

Tensorflow data uses protos. If you are loading from a file there are helpers for this. Since we are using generated data, this code populates a proto.

In [8]:
# This is an example of a proto that converts the samples and labels into
# tensors usable by tensorflow.
class ExampleProcessor:
    def create_example_with_label(self, feature: numpy.float32,
                             label: numpy.float32)-> tf.train.Example:
        return tf.train.Example(
            features=tf.train.Features(
                  feature={'x': self.create_feature(feature),
                           'y' : self.create_feature(label)
                  }))

    def create_example(self, feature: numpy.float32):
        return tf.train.Example(
            features=tf.train.Features(
                  feature={'x' : self.create_feature(feature)})
            )

    def create_feature(self, element: numpy.float32):
        return tf.train.Feature(float_list=tf.train.FloatList(value=[element]))

# Create a labeled example file for 5 times table.

example_five_times_table = 'example_five_times_table.tfrecord'

with tf.io.TFRecordWriter(example_five_times_table) as writer:
  for i in zip(x, y):
    example = ExampleProcessor().create_example_with_label(
        feature=i[0], label=i[1])
    writer.write(example.SerializeToString())

# Create a file containing the values to predict.

predict_values_five_times_table = 'predict_values_five_times_table.tfrecord'

with tf.io.TFRecordWriter(predict_values_five_times_table) as writer:
  for i in value_to_predict:
    example = ExampleProcessor().create_example(feature=i)
    writer.write(example.SerializeToString())

### Fit The Model

This example builds a model. Since RunInference requires pretrained models, this segment builds a usable model.

In [9]:
RAW_DATA_TRAIN_SPEC = {
'x': tf.io.FixedLenFeature([], tf.float32),
'y': tf.io.FixedLenFeature([], tf.float32)
}

dataset = tf.data.TFRecordDataset(example_five_times_table)
dataset = dataset.map(lambda e : tf.io.parse_example(e, RAW_DATA_TRAIN_SPEC))
dataset = dataset.map(lambda t : (t['x'], t['y']))
dataset = dataset.batch(100)
dataset = dataset.repeat()

model.fit(dataset, epochs=500, steps_per_epoch=1, verbose=0)

### Save the Model

In [12]:
RAW_DATA_PREDICT_SPEC = {
'x': tf.io.FixedLenFeature([], tf.float32),
}

@tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string , name='examples')])
def serve_tf_examples_fn(serialized_tf_examples):
  """Returns the output to be used in the serving signature."""
  features = tf.io.parse_example(serialized_tf_examples, RAW_DATA_PREDICT_SPEC)
  return model(features, training=False)

signature = {'serving_default': serve_tf_examples_fn}

tf.keras.models.save_model(model, save_model_dir_multiply, signatures=signature)

## Run the Pipeline

FormatOutput demonstrates how to extract values from the output protos.

CreateModelHandler demonstrates the model handler that needs to be passed into beams RunInference API.

In [13]:
from tfx_bsl.public.beam.run_inference import CreateModelHandler

class FormatOutput(beam.DoFn):
    def process(self, element: prediction_log_pb2.PredictionLog):
        predict_log = element.predict_log
        input_value = tf.train.Example.FromString(predict_log.request.inputs['examples'].string_val[0])
        input_float_value = input_value.features.feature['x'].float_list.value
        output_value = predict_log.response.outputs
        output_float_value = output_value['output_0'].float_val
        yield (f"example is {input_float_value} prediction is {output_float_value}")

tfexample_beam_record = tfx_bsl.public.tfxio.TFExampleRecord(file_pattern=predict_values_five_times_table)
saved_model_spec = model_spec_pb2.SavedModelSpec(model_path=save_model_dir_multiply)
inferece_spec_type = model_spec_pb2.InferenceSpecType(saved_model_spec=saved_model_spec)
model_handler = CreateModelHandler(inferece_spec_type)
with beam.Pipeline() as p:
    _ = (p | tfexample_beam_record.RawRecordBeamSource() 
           | RunInference(model_handler)
           | beam.ParDo(FormatOutput())
           | beam.Map(print)
        )

example is [20.0] prediction is [100.00492858886719]
example is [40.0] prediction is [200.00970458984375]
example is [60.0] prediction is [300.01446533203125]
example is [90.0] prediction is [450.0216064453125]
